# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-0.6904,  0.6536],
         [ 1.1455, -0.5581],
         [-1.2724, -0.7850],
         [ 0.8606, -0.0378],
         [-0.5898, -0.4856],
         [ 1.2049,  1.5855],
         [-0.3981,  1.9617],
         [-0.0212, -0.7124],
         [-0.0486,  0.6529],
         [-0.6305, -0.3003]]),
 tensor([[ 0.5962],
         [ 8.4040],
         [ 4.3348],
         [ 6.0500],
         [ 4.6911],
         [ 1.2281],
         [-3.2724],
         [ 6.5691],
         [ 1.8739],
         [ 3.9565]])]

使用框架的预定义好的层

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [8]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000399
epoch 2, loss 0.000099
epoch 3, loss 0.000098


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0002, -0.0003])
b的估计误差： tensor([-0.0005])
